In [53]:
from pymongo import MongoClient
from pathlib import Path
import os
from pprint import pprint
import nltk
import matplotlib.pyplot as plt
import pprint

In [54]:
client = MongoClient('mongodb://localhost:27017')
testDb = client.testDb

In [55]:
patients = testDb.patients
sessions = testDb.sessions
reports = testDb.reports

In [56]:
def fileName(filePath):
    return (os.path.splitext(os.path.split(filePath)[1]))[0]

def fileExt(filePath):
    try:
        return (os.path.splitext(os.path.split(filePath)[1]))[1]
    except:
        return "Error"

In [68]:
p1 = os.listdir("v1.2.0/eval")
print(p1)
print(os.path.isdir(p1[0]))
print(os.path.isfile(p1[1]))
pathString = ("/"+str(p1[0]))
print(pathString)
patientDict = {}
fileNames = []
maxFileSize = 0 
maxFileName = ""
for root,dirs,files in os.walk("v1.2.0"):
    #print ("root: ",root)
    #print("\tdirs: ",dirs)
    #print("\t\t",str(files))
        
    if len(files)>0:
        for file in files:
            fileNames.append(str(root)+"/"+str(file))
            if os.path.getsize((str(root)+"/"+file)) > maxFileSize:
                maxFileSize = os.path.getsize((str(root)+"/"+file))
                maxFileName = (str(root)+"/"+file)
            else:
                continue
            
            
        #print ("\t\tFile: "+files[3])
        
print (maxFileName," : ",maxFileSize)
print (os.path.splitext(maxFileName))
print (os.path.split(maxFileName))
print (os.path.splitext(os.path.split(maxFileName)[1]))
print (fileName(maxFileName))
print (fileExt(maxFileName))

['03_tcp_ar_a', '02_tcp_le', '.DS_Store', '01_tcp_ar']
False
False
/03_tcp_ar_a
v1.2.0/eval/01_tcp_ar/00001278/s002_2003_11_17/00001278_s002_t001.edf  :  67899648
('v1.2.0/eval/01_tcp_ar/00001278/s002_2003_11_17/00001278_s002_t001', '.edf')
('v1.2.0/eval/01_tcp_ar/00001278/s002_2003_11_17', '00001278_s002_t001.edf')
('00001278_s002_t001', '.edf')
00001278_s002_t001
.edf


In [108]:
def addFile(filepath):
    splitFile = filepath.split('/')
    if len(splitFile)<6:
        print("File path too short")
    else:
        patientId = splitFile[3]
        currentFile = splitFile[5] #changed from 5 to 4
        currentSessionNumber = splitFile[4].split('_')[0]
        currentFileSplit = currentFile.split('_')
        currentFileExt = str(fileExt(currentFile))
        sessionNumber = currentFileSplit[0] #Changed from 1 to 0
        sessionId = str(patientId)+str(currentSessionNumber)
        print("\tFile Path: ",filepath)
        print("\t\tCurrentFile: ",currentFile)
        print("\t\tCurrentFileExt: ",currentFileExt)
        print("\t\tSessionId: ",sessionId)
        dbSession = sessions.find_one({"sessionId":sessionId})
        
        if dbSession == None:
            print("Found a new session")
            sessions.insert_one({"sessionId":sessionId,"patientId":patientId})
        else:
            print("Found this session\n")  
        if currentFileExt == ".txt":
            try:
                with open(str(files),'r') as reportFile:
                    rText = reportFile.readlines()  
                try:
                    sessions.update_one({"sessionId":sessionId},
                                       {"$set":{"reportText":rText}})
                except:
                    print("Could not add report text")     
            except UnicodeDecodeError:
                print("Could not open text file")
                sessions.update_one({"sessionId":sessionId},
                                       {"$set":{"reportText":"Unopenable"}})
        elif currentFileExt  == ".edf":
            try:
                updateEdf = dbSession['edfList']
                updateEdf.append(filepath)
                sessions.update_one({"sessionId":sessionId},{"$set":{"edfList":updateEdf}})    
            except:
                #print ("No edf") 
                sessions.update_one({"sessionId":sessionId},{"$set":{"edfList":[filepath]}})
        elif currentFileExt == ".lbl":
            try:
                updateLbl = dbSession['lblList']
                updateLbl.append(filepath)
                print("\n\nUPDATING LABL LIST-------------")
                print(updateLbl)
                sessions.update_one({"sessionId":sessionId},{"$set":{"lblList":updateLbl}})
            except:
                sessions.update_one({"sessionId":sessionId},{"$set":{"lblList":[filepath]}})
                
        elif currentFileExt == ".tse":
            try:
                updateTse = dbSession['tseList']
                updateTse.append(filepath)
                sessions.update_one({"sessionId":sessionId},{"$set":{"tseList":updateTse}})
            except:
                sessions.update_one({"sessionId":sessionId},{"$set":{"tseList":[filepath]}})

In [109]:
#print(fileNames[0:30])
testDb.patients.delete_many({})
testDb.sessions.delete_many({})
testDb.reports.delete_many({})

for files in fileNames:
    addFile(files)
    

File path too short
File path too short
File path too short
File path too short
	File Path:  v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002.txt
		CurrentFile:  00005943_s002.txt
		CurrentFileExt:  .txt
		SessionId:  00005943s002
Found a new session
	File Path:  v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t002.lbl_bi
		CurrentFile:  00005943_s002_t002.lbl_bi
		CurrentFileExt:  .lbl_bi
		SessionId:  00005943s002
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t001.tse_bi
		CurrentFile:  00005943_s002_t001.tse_bi
		CurrentFileExt:  .tse_bi
		SessionId:  00005943s002
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t000.lbl_bi
		CurrentFile:  00005943_s002_t000.lbl_bi
		CurrentFileExt:  .lbl_bi
		SessionId:  00005943s002
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00005943/s002_2009_06_28/00005943_s002_t003.tse_bi
		CurrentFile:  00005943_s002_t003.

		CurrentFileExt:  .tse_bi
		SessionId:  00004087s003
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00004087/s003_2007_07_24/00004087_s003_t003.edf
		CurrentFile:  00004087_s003_t003.edf
		CurrentFileExt:  .edf
		SessionId:  00004087s003
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00004087/s003_2007_07_24/00004087_s003_t000.lbl_bi
		CurrentFile:  00004087_s003_t000.lbl_bi
		CurrentFileExt:  .lbl_bi
		SessionId:  00004087s003
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00004087/s003_2007_07_24/00004087_s003_t001.edf
		CurrentFile:  00004087_s003_t001.edf
		CurrentFileExt:  .edf
		SessionId:  00004087s003
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00004087/s003_2007_07_24/00004087_s003_t000.edf
		CurrentFile:  00004087_s003_t000.edf
		CurrentFileExt:  .edf
		SessionId:  00004087s003
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00004087/s003_2007_07_24/00004087_s003.txt
		CurrentFile:  00004087_s003.txt
		CurrentFileExt:  .txt

	File Path:  v1.2.0/eval/03_tcp_ar_a/00002297/s004_2007_10_23/00002297_s004.txt
		CurrentFile:  00002297_s004.txt
		CurrentFileExt:  .txt
		SessionId:  00002297s004
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00002297/s004_2007_10_23/00002297_s004_t001.tse
		CurrentFile:  00002297_s004_t001.tse
		CurrentFileExt:  .tse
		SessionId:  00002297s004
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00002297/s004_2007_10_23/00002297_s004_t001.lbl
		CurrentFile:  00002297_s004_t001.lbl
		CurrentFileExt:  .lbl
		SessionId:  00002297s004
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00002297/s004_2007_10_23/00002297_s004_t000.lbl
		CurrentFile:  00002297_s004_t000.lbl
		CurrentFileExt:  .lbl
		SessionId:  00002297s004
Found this session



UPDATING LABL LIST-------------
['v1.2.0/eval/03_tcp_ar_a/00002297/s004_2007_10_23/00002297_s004_t001.lbl', 'v1.2.0/eval/03_tcp_ar_a/00002297/s004_2007_10_23/00002297_s004_t000.lbl']
	File Path:  v1.2.0/eval/03_tcp_ar_a/00002297

	File Path:  v1.2.0/eval/03_tcp_ar_a/00006546/s012_2011_02_17/00006546_s012_t003.edf
		CurrentFile:  00006546_s012_t003.edf
		CurrentFileExt:  .edf
		SessionId:  00006546s012
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00006546/s012_2011_02_17/00006546_s012_t004.lbl
		CurrentFile:  00006546_s012_t004.lbl
		CurrentFileExt:  .lbl
		SessionId:  00006546s012
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00006546/s012_2011_02_17/00006546_s012_t004.tse
		CurrentFile:  00006546_s012_t004.tse
		CurrentFileExt:  .tse
		SessionId:  00006546s012
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00006546/s012_2011_02_17/00006546_s012_t001.lbl_bi
		CurrentFile:  00006546_s012_t001.lbl_bi
		CurrentFileExt:  .lbl_bi
		SessionId:  00006546s012
Found this session

	File Path:  v1.2.0/eval/03_tcp_ar_a/00006546/s012_2011_02_17/00006546_s012_t002.tse_bi
		CurrentFile:  00006546_s012_t002.tse_bi
		CurrentFileExt:  .tse_bi
		SessionId:  00006546s012
Found this session

	File P

		CurrentFileExt:  .tse_bi
		SessionId:  00003635s001
Found this session

	File Path:  v1.2.0/eval/02_tcp_le/00004087/s001_2007_06_26/00004087_s001_t000.edf
		CurrentFile:  00004087_s001_t000.edf
		CurrentFileExt:  .edf
		SessionId:  00004087s001
Found a new session
	File Path:  v1.2.0/eval/02_tcp_le/00004087/s001_2007_06_26/00004087_s001_t000.tse
		CurrentFile:  00004087_s001_t000.tse
		CurrentFileExt:  .tse
		SessionId:  00004087s001
Found this session

	File Path:  v1.2.0/eval/02_tcp_le/00004087/s001_2007_06_26/00004087_s001_t000.lbl
		CurrentFile:  00004087_s001_t000.lbl
		CurrentFileExt:  .lbl
		SessionId:  00004087s001
Found this session

	File Path:  v1.2.0/eval/02_tcp_le/00004087/s001_2007_06_26/00004087_s001_t000.lbl_bi
		CurrentFile:  00004087_s001_t000.lbl_bi
		CurrentFileExt:  .lbl_bi
		SessionId:  00004087s001
Found this session

	File Path:  v1.2.0/eval/02_tcp_le/00004087/s001_2007_06_26/00004087_s001.txt
		CurrentFile:  00004087_s001.txt
		CurrentFileExt:  .txt
		Session

...

	File Path:  v1.2.0/eval/01_tcp_ar/00008512/s003_2012_01_10/00008512_s003_t002.edf
		CurrentFile:  00008512_s003_t002.edf
		CurrentFileExt:  .edf
		SessionId:  00008512s003
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00008512/s003_2012_01_10/00008512_s003_t002.tse_bi
		CurrentFile:  00008512_s003_t002.tse_bi
		CurrentFileExt:  .tse_bi
		SessionId:  00008512s003
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00008512/s003_2012_01_10/00008512_s003_t001.lbl_bi
		CurrentFile:  00008512_s003_t001.lbl_bi
		CurrentFileExt:  .lbl_bi
		SessionId:  00008512s003
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00008512/s003_2012_01_10/00008512_s003_t000.tse_bi
		CurrentFile:  00008512_s003_t000.tse_bi
		CurrentFileExt:  .tse_bi
		SessionId:  00008512s003
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00008512/s003_2012_01_10/00008512_s003_t003.lbl_bi
		CurrentFile:  00008512_s003_t003.lbl_bi
		CurrentFileExt:  .lbl_bi
		SessionId:  00008512s003
Found this session


	File Path:  v1.2.0/eval/01_tcp_ar/00006059/s005_2012_09_07/00006059_s005_t002.lbl
		CurrentFile:  00006059_s005_t002.lbl
		CurrentFileExt:  .lbl
		SessionId:  00006059s005
Found this session



UPDATING LABL LIST-------------
['v1.2.0/eval/01_tcp_ar/00006059/s005_2012_09_07/00006059_s005_t001.lbl', 'v1.2.0/eval/01_tcp_ar/00006059/s005_2012_09_07/00006059_s005_t000.lbl', 'v1.2.0/eval/01_tcp_ar/00006059/s005_2012_09_07/00006059_s005_t002.lbl']
	File Path:  v1.2.0/eval/01_tcp_ar/00006059/s005_2012_09_07/00006059_s005_t002.tse
		CurrentFile:  00006059_s005_t002.tse
		CurrentFileExt:  .tse
		SessionId:  00006059s005
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00006059/s005_2012_09_07/00006059_s005_t002.edf
		CurrentFile:  00006059_s005_t002.edf
		CurrentFileExt:  .edf
		SessionId:  00006059s005
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00006059/s005_2012_09_07/00006059_s005_t001.edf
		CurrentFile:  00006059_s005_t001.edf
		CurrentFileExt:  .edf
		SessionId:  0000605

		CurrentFileExt:  .edf
		SessionId:  00006546s021
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00006546/s021_2012_02_22/00006546_s021_t005.edf
		CurrentFile:  00006546_s021_t005.edf
		CurrentFileExt:  .edf
		SessionId:  00006546s021
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00006546/s021_2012_02_22/00006546_s021_t004.edf
		CurrentFile:  00006546_s021_t004.edf
		CurrentFileExt:  .edf
		SessionId:  00006546s021
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00006546/s021_2012_02_22/00006546_s021_t010.edf
		CurrentFile:  00006546_s021_t010.edf
		CurrentFileExt:  .edf
		SessionId:  00006546s021
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00006546/s021_2012_02_22/00006546_s021_t003.tse_bi
		CurrentFile:  00006546_s021_t003.tse_bi
		CurrentFileExt:  .tse_bi
		SessionId:  00006546s021
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00006546/s021_2012_02_22/00006546_s021_t006.edf
		CurrentFile:  00006546_s021_t006.edf
		CurrentFileExt:  .edf
		

		CurrentFile:  00010022_s004_t009.tse
		CurrentFileExt:  .tse
		SessionId:  00010022s004
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00010022/s004_2013_02_12/00010022_s004_t011.tse_bi
		CurrentFile:  00010022_s004_t011.tse_bi
		CurrentFileExt:  .tse_bi
		SessionId:  00010022s004
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00010022/s004_2013_02_12/00010022_s004_t009.lbl
		CurrentFile:  00010022_s004_t009.lbl
		CurrentFileExt:  .lbl
		SessionId:  00010022s004
Found this session



UPDATING LABL LIST-------------
['v1.2.0/eval/01_tcp_ar/00010022/s004_2013_02_12/00010022_s004_t001.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s004_2013_02_12/00010022_s004_t000.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s004_2013_02_12/00010022_s004_t002.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s004_2013_02_12/00010022_s004_t003.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s004_2013_02_12/00010022_s004_t007.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s004_2013_02_12/00010022_s004_t006.lbl', 'v1.2.0/eval/01_tcp_ar

		CurrentFile:  00010022_s003_t014.lbl_bi
		CurrentFileExt:  .lbl_bi
		SessionId:  00010022s003
Found this session

	File Path:  v1.2.0/eval/01_tcp_ar/00010022/s003_2013_02_11/00010022_s003_t000.lbl
		CurrentFile:  00010022_s003_t000.lbl
		CurrentFileExt:  .lbl
		SessionId:  00010022s003
Found this session



UPDATING LABL LIST-------------
['v1.2.0/eval/01_tcp_ar/00010022/s003_2013_02_11/00010022_s003_t013.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s003_2013_02_11/00010022_s003_t012.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s003_2013_02_11/00010022_s003_t006.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s003_2013_02_11/00010022_s003_t010.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s003_2013_02_11/00010022_s003_t004.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s003_2013_02_11/00010022_s003_t005.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s003_2013_02_11/00010022_s003_t011.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s003_2013_02_11/00010022_s003_t015.lbl', 'v1.2.0/eval/01_tcp_ar/00010022/s003_2013_02_11/00010022_s003_t001.lbl',

In [114]:
sessionFound  = sessions.find_one({})
#print(sessionFound.count())
#print(sessionFound)
allSessions = sessions.find({'patientId':'00006546'})
print(allSessions.count())

for x in allSessions:
    print(x['sessionId'])

18
00006546s011
00006546s010
00006546s015
00006546s013
00006546s016
00006546s012
00006546s009
00006546s026
00006546s005
00006546s022
00006546s020
00006546s014
00006546s024
00006546s021
00006546s025
00006546s018
00006546s001
00006546s023


In [52]:
patientDbPrint = patients.find({})
sessionsDb = sessions.find({})
sessionList = []
for session in sessionsDb:
    if len(session.keys())>4:
        sessionList.append(session)
    #print(session)
    
#reportText1 = sessionList[0]['ReportText']

reportTextString = " ".join(reportText1)

#print(reportTextString)

reportTextStringList = reportTextString.split(" ")

fDist = nltk.FreqDist(reportTextStringList)
print(fDist.items())
#print(reportTextStringList)

'''for x in reportText1:
    print("\t"+x)
    print("\n")
    

for patient in patientDbPrint:
    print(patient)
'''

dict_items([('CLINICAL', 3), ('HISTORY:', 1), ('50', 2), ('year', 1), ('old', 1), ('right', 2), ('handed', 1), ('woman', 1), ('status', 7), ('post', 1), ('left', 13), ('MCA', 1), ('aneurysm', 1), ('repaired', 1), ('in', 25), ('2003', 1), ('with', 26), ('annual', 1), ('exacerbation', 1), ('of', 36), ('complex', 3), ('partial', 2), ('seizures', 9), ('and', 24), ('low', 1), ('serum', 1), ('levels.\n', 1), ('MEDICATIONS:', 1), ('Depakote,', 1), ('Keppra,\n', 1), ('INTRODUCTION:', 3), ('Digital', 1), ('video', 3), ('EEG', 11), ('was', 9), ('performed', 3), ('at', 14), ('bedside', 2), ('using', 3), ('standard', 3), ('10-20', 3), ('system', 3), ('electrode', 3), ('placement', 2), ('1', 3), ('channel', 2), ('EKG.', 2), ('This', 6), ('is', 38), ('a', 20), ('prolonged', 1), ('study', 1), ('for', 5), ('patient', 21), ('who', 1), ('appeared', 1), ('to', 12), ('have', 3), ('had', 1), ('subclinical', 1), ('seizure.', 2), ('She', 4), ('confused', 1), ('during', 5), ('the', 89), ('somewhat', 3), ('let

'for x in reportText1:\n    print("\t"+x)\n    print("\n")\n    \n\nfor patient in patientDbPrint:\n    print(patient)\n'

In [ ]:
#fDist.most_common(30)
mostCommon = fDist.most_common(20)
xAxisVals = []
yAxisVals = []

plt.figure(figsize=(10,10))
for x in mostCommon:
    xAxisVals.append(x[0])
    yAxisVals.append(x[1])
plt.bar(xAxisVals,yAxisVals)
plt.show()

In [ ]:
'''
    splitFile = files.split("/")
    #print(splitFile)
    if len(splitFile)<6:
        pass
    else:
        patientId = splitFile[3]
        
        currentFile = splitFile[5]
        currentFileSplit = currentFile.split('_')
        sessionNumber = currentFileSplit[1]
        patientId = currentFileSplit[0]
        sessionId = str(patientId)+str(sessionNumber)
        #print("\t\t\t splitFile: ",currentFileSplit)
        #print("\t\tFilePath: "+files)
        currentFileExt = str(fileExt(currentFile))
        #print("\tCurrent File: "+currentFile) 
        if currentSession != sessionId: 
            currentSession = sessionId
            if bool(sessionEntry) == True:
                #Insert session into patient session list
                sessionEntry['edfList'] = edfList
                sessionEntry['lblList'] = lblList
                sessionEntry['tseList'] = tseList
                #print("CurrentSession: "+currentSession)
                
                if currentFileExt == ".txt":
                    sessionEntry['Report'] = currentFile
                    try:
                        with open(str(files),'r') as reportFile:
                            rText = reportFile.readlines()      
                        sessionEntry['ReportText'] = rText
                    except UnicodeDecodeError:
                        sessionEntry['ReportText'] = "Error Opening Report"
                elif currentFileExt  == ".edf":
                    edfList.append(currentFile)
                elif currentFileExt == ".lbl":
                    lblList.append(currentFile)
                elif currentFileExt == ".tse":
                    tseList.append(currentFile)
                    print("Adding Session!!!")
                    sessions.insert_one(sessionEntry)
                    sessionEntry = {}
            else:            #Found a new session
                print("Found an empty Session")
                print(sessionEntry)
                sessionEntry = {'sessionId':sessionId}
                sessionEntry['patientId'] = patientId
                lblList = []
                edfList = []
                tseList = []
                tseBiList = []
                lblBiList = []
                if currentFileExt == ".txt":
                    sessionEntry['Report'] = currentFile
                    try:
                        with open(str(files),'r') as reportFile:
                            rText = reportFile.readlines()      
                        sessionEntry['ReportText'] = rText
                    except UnicodeDecodeError:
                        sessionEntry['ReportText'] = "Error Opening Report"
                elif currentFileExt  == ".edf":
                    edfList.append(currentFile)
                elif currentFileExt == ".lbl":
                    lblList.append(currentFile)
                elif currentFileExt == ".tse":
                    tseList.append(currentFile)
            
        else:
            if currentFileExt == ".txt":
                sessionEntry['Report'] = currentFile
                try:
                    with open(str(files),'r') as reportFile:
                        rText = reportFile.readlines()      
                    sessionEntry['ReportText'] = rText
                except UnicodeDecodeError:
                    sessionEntry['ReportText'] = "Error Opening Report"
            elif currentFileExt  == ".edf":
                edfList.append(currentFile)
            elif currentFileExt == ".lbl":
                lblList.append(currentFile)
            elif currentFileExt == ".tse":
                tseList.append(currentFile)
                
            if patientId in patientList:
                #Add Session Id to patient sessions
                #Find patient entry, add session 
                continue
            else:
                #Add sessionId to patient session
                patients.insert_one({'patientId':patientId})
                patientList.append(patientId)
# break
            if sessionId in sessionList:
                continue
            else:
                sessionList.append(sessionId)
                if patientId in patientLi[=st:
                    sessions.update_one({"patientId":patientId},{'$push':{"sessionsId":sessionId}})
                else:
                    sessions.insert_one({{'patientId':patientId},{'sessionId':[sessionId]}})
            
        currentSession = sessionId
#print (patientList)
print (len(patientList))

'''